In [ ]:
import tensorflow as tf
import tensorflow_hub as hub


In [ ]:
# Load the USE model
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
use_model = hub.load(module_url)

In [ ]:
import pickle

In [ ]:
def save_arr(sentence_embeddings,name):
    with open(name, 'wb') as f:
        pickle.dump(sentence_embeddings, f)

In [ ]:
def read_text(file_path):

    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()

    for i in range(len(lines)):
        lines[i]=lines[i].strip()

    return lines

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
import numpy as np

In [ ]:
def knn(e,sentence_embeddings,k=100):
    nbrs = NearestNeighbors(n_neighbors=k)
    nbrs.fit(sentence_embeddings)
    distances, indices = nbrs.kneighbors([e])
    indices=np.squeeze(indices)
    return indices

In [ ]:
def s_score(i,i1,i2):
    common = np.intersect1d(np.intersect1d(i, i1), i2)
    s = len(common)

    print("Common elements:", common)
    print("Number of common elements:", s)
    return common

In [ ]:
def indices(s,sentence_embeddings):
    print(s)
    e = np.squeeze(use_model([s]))
    i = knn(e,sentence_embeddings)
    # print(i)
    return i

In [ ]:
sentences =read_text("generic.txt")
sentence_embeddings = use_model(sentences)

# sentence_embeddings = torch.tensor(sentence_embeddings.numpy())
# save_arr(sentence_embeddings,'bbc_e.pkl')

In [ ]:
print(sentence_embeddings.shape)

(340290, 512)


In [ ]:
use_model(["hi im a small child"]).shape

TensorShape([1, 512])

In [ ]:
s="i couldnt leave because it was raining very heavily"
i = indices(s,sentence_embeddings)
i1= indices("i couldnt leave because it was showering very heavily",sentence_embeddings)
i2= indices("i couldnt leave because it was raining very he strongly",sentence_embeddings)

# s="i couldnt leave because it was raining cats and dogs"
# i = indices(s,sentence_embeddings)
# i1= indices("i couldnt leave because it was showering cats and dogs",sentence_embeddings)
# i2= indices("i couldnt leave because it was raining cats and wolves",sentence_embeddings)

# for x in i:
#     print(sentences[x])
#     print()

# for x in i1:
#     print(sentences[x])
#     print()

# for x in i2:
#     print(sentences[x])
#     print()

y= s_score(i,i1,i2)

print(i)
print(i1)

for x in y:
    print(sentences[x])
    print()

i couldnt leave because it was raining very heavily
i couldnt leave because it was showering very heavily
i couldnt leave because it was raining very he strongly
Common elements: [252988 253137]
Number of common elements: 2
[328898 114154  92240 307428 253076 225867  92241 253136 253138 253104
 233177 253164 294471 313575 253048 253254  92828 253050 253099 253096
 253149 253142 253167 196089 253161 253130 253075 253425 294458 294519
 290706 253074 253147 253089 238822 253367 253041 253055 333129  17435
 253106 253135  20460 253097 253391 144062  76068  57670 253047  97768
 253070 294477 253146 253072 233121 173557  61238 253231 253090 330036
 100772 263332 328066 253061 325324 253060  17368 252988 253144  87510
 253065 226423 253084 253035  92243 242643 253109 253336 167618 253137
 253394   8964 233141 317239 253033 253085 205667 253378   1595 253108
 253140 147561 253105 253155 252976  17387  61183 300136 192587 170960]
[ 89764  23231  26685 225881 276628  70166 239892 276611 335347 2

In [ ]:
!pip install annoy

In [ ]:
!pip install sentence-transformers

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
from annoy import AnnoyIndex

In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
sentences[::1000]

In [ ]:
embeddings = model.encode(sentences[::10])

In [ ]:
embeddings.shape

(34029, 384)

In [ ]:
embedding_dim = embeddings.shape[1]

annoy_index = AnnoyIndex(embedding_dim, metric='angular')

for i, embedding in enumerate(embeddings):
    annoy_index.add_item(i, embedding)

annoy_index.build(n_trees=10)

k = 100
ind = annoy_index.get_nns_by_vector(model.encode("i couldnt leave because it was raining cats and dogs"), k)
i1 = annoy_index.get_nns_by_vector(model.encode("i couldnt leave because it was showering cats and dogs"), k)
i2 = annoy_index.get_nns_by_vector(model.encode("i couldnt leave because it was raining cats and wolves"), k)

# ind = annoy_index.get_nns_by_vector(model.encode("i couldnt leave because it was raining very heavily"), k)
# i1 = annoy_index.get_nns_by_vector(model.encode("i couldnt leave because it was showering very heavily"), k)
# i2 = annoy_index.get_nns_by_vector(model.encode("i couldnt leave because it was raining very he strongly"), k)

nearest_neighbors = [sentences[::10][i] for i in ind]
nearest_neighbors_1 = [sentences[::10][i] for i in i1]
nearest_neighbors_2 = [sentences[::10][i] for i in i2]

# for i, neighbor in enumerate(nearest_neighbors):
#     print(f"Neighbor {i + 1}: {neighbor}")

# for i, neighbor in enumerate(nearest_neighbors_1):
#     print(f"Neighbor {i + 1}: {neighbor}")

# for i, neighbor in enumerate(nearest_neighbors_2):
#     print(f"Neighbor {i + 1}: {neighbor}")

y= s_score(ind,i1,i2)
print()

for x in y:
    print(sentences[::10][x])
    print()

Common elements: [ 1139  1331  4862  4868  4869  4870  4872  4875  4876  4878  4883  4884
  4887  4889  4893  4895  4898  4901  4906  4914  4922  4926  4927  4932
  5518  6014  8988  9144 11315 11316 11808 15675 15716 16634 16636 19747
 19760 22098 22883 25085 30138 32306 33190 33621 33939]
Number of common elements: 45

Animals dislike the smell of perfume odours on their coats.

Some animals just keep that smell, it fades after a while.

Cats are little people in fur coats.

Cats are small mammals.

Cats are territorial animals.

Cats are used for pets.

Cats bury their feces to cover their trails from predators.

Cats can have diarrhea.

Cats can smell mice from over half a mile away.

Cats cool themselves by rapid breathing and by licking their coats.

Cats get Toxoplasmosis by eating rodents or uncooked meat of any kind.

Cats hate loud noises.

Cats have connotations of witch-magic, cosy cottages or revels by night, dogs are loyal and loving.

Cats have litter type preferences.

